#### Creating a simple array with random items

In [1]:
import numpy as np 

np.set_printoptions(suppress = True)

# Generatinf random distribution "tensors"
params = np.random.uniform(low = -20, high = 200, size = 15)

In [2]:
# modifing for easy debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

In [3]:
params = np.round(params,4)
params

array([199.9341,  -9.1758,   0.    ,   7.1529, 123.2222, 101.2439,
       182.7486,  88.2835,  58.6505,  -8.1758,  -4.9222, 159.3742,
       110.0036, 159.674 , 198.9341])

### Function for clamping

In [4]:
def clamping(params_arr, lower_bound, upper_bound):
    params_arr[params_arr < lower_bound] = lower_bound
    params_arr[params_arr > upper_bound] = upper_bound
    return params_arr

### Function for asymmetric quantization and dequantization

In [11]:
def asymmetric_quantization(params, bits):
    alpha = np.max(params) # the largest value in our "tensor"
    beta = np.min(params) # smallest value in our "tensor"
    scale = (alpha - beta) / (2**bits - 1) # here wa can also use min/max scaler
    zero_point = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, (2**bits - 1)
    # Quantization 
    quantized = clamping(np.round(params/scale + zero_point), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero_point

def asymmetric_dequantization(params_q, scale, zero_point):
    return scale * (params_q - zero_point)

### Function for symmetric quantization and dequantization

In [37]:
def symmetric_quantization(params, bits):
    alpha = np.max(np.abs(params)) # the max absolute value
    scale = alpha / (2**(bits-1) - 1)
    lower_bound, upper_bound = -2**(bits - 1) - 1, 2**(bits-1) - 1
    # Quantization 
    quantized = clamping(np.round(params/scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def symmetric_dequantization(params_q, scale):
    return scale * params_q

### Quantization error

In [23]:
def quantization_error(params, params_q):
    # we can calculate any for of loss here. ** MSE **
    return np.mean((params - params_q)**2)

### Seeing how it plays out

In [24]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[199.93  -9.18   0.     7.15 123.22 101.24 182.75  88.28  58.65  -8.18
  -4.92 159.37 110.   159.67 198.93]

Asymmetric scale: 0.8200388235294118, zero: 11.0
[255   0  11  20 161 134 234 119  83   1   5 205 145 206 254]

Symmetric scale: 1.5742842519685039
[127  -6   0   5  78  64 116  56  37  -5  -3 101  70 101 126]


In [25]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantization(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantization(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[199.93  -9.18   0.     7.15 123.22 101.24 182.75  88.28  58.65  -8.18
  -4.92 159.37 110.   159.67 198.93]

Dequantize Asymmetric:
[200.09  -9.02   0.     7.38 123.01 100.86 182.87  88.56  59.04  -8.2
  -4.92 159.09 109.89 159.91 199.27]

Dequantize Symmetric:
[199.93  -9.45   0.     7.87 122.79 100.75 182.62  88.16  58.25  -7.87
  -4.72 159.   110.2  159.   198.36]


### Error calculation

In [28]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.05
   Symmetric error: 0.15


### Playing around

In [30]:
# Generate 10 random positive values and 10 random negative values
positive_values = np.random.uniform(low=0, high=20, size=10)
negative_values = np.random.uniform(low=-20, high=0, size=10)

# Concatenate the arrays to simulate symmetry
params_sy = np.concatenate((negative_values, positive_values))

# For an additional step, you can shuffle the values if needed
# np.random.shuffle(params)

print("Symmetric tensor-like array:")
print(params_sy)

Symmetric tensor-like array:
[-16.88845729 -14.73619101  -9.84795077  -2.16624371 -13.58845586
 -13.89295657  -5.13396663  -7.43403557  -2.02378329  -4.6139564
  13.66297264  12.94906411   4.74121694   5.52796603   5.09095534
   8.92738701   7.37523507   2.56579524  16.80063994  13.47113548]


In [31]:
(symmetric_q, symmetric_scale) = symmetric_quantization(params_sy, 8)

In [32]:
print(f'Original:')
print(np.round(params_sy, 2))
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[-16.89 -14.74  -9.85  -2.17 -13.59 -13.89  -5.13  -7.43  -2.02  -4.61
  13.66  12.95   4.74   5.53   5.09   8.93   7.38   2.57  16.8   13.47]

Symmetric scale: 0.13297997865189634
[-127 -111  -74  -16 -102 -104  -39  -56  -15  -35  103   97   36   42
   38   67   55   19  126  101]


In [33]:
params_deq_symmetric = symmetric_dequantization(symmetric_q, symmetric_scale)
print(f'Original:')
print(np.round(params_sy, 2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[-16.89 -14.74  -9.85  -2.17 -13.59 -13.89  -5.13  -7.43  -2.02  -4.61
  13.66  12.95   4.74   5.53   5.09   8.93   7.38   2.57  16.8   13.47]

Dequantize Symmetric:
[-16.89 -14.76  -9.84  -2.13 -13.56 -13.83  -5.19  -7.45  -1.99  -4.65
  13.7   12.9    4.79   5.59   5.05   8.91   7.31   2.53  16.76  13.43]


In [36]:
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params_sy, params_deq_symmetric), 5)}')

   Symmetric error: 0.00159


#### Because of the nature of the data we can seehow the error is very low. 